# Tarea 2: Encoder-Decoder Architectures
## Para el curso de Transformers & Diffusers
### Alan García Zermeño, 3/10/2023

## Ejercicio 2: Traductor Encoder-Decoder
### Usaremos los datos de https://tatoeba.org/es/, en especial los de Inglés-Alemán y los de Inglés-Español que comparto aquí: https://drive.google.com/drive/folders/1nK76bkQ8bXEk44vgpZeFsxkXz9-Zwsy5?usp=sharing. Se compone por secuencias de pares Lang1-Lang2.

In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from io import open
import unicodedata
import re
import random
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from tqdm.notebook import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
BOS_token = 0 #Token de inicio de sentencia
EOS_token = 1 #Token de final de sentencia

#Clase para vocabulario de lenguaje
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "BOS", 1: "EOS"}
        self.n_words = 2

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:
MAX_LENGTH = 15 #Filtro por longitud de secuencia máxima

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filter(pairs):
    return [pair[:2] for pair in pairs if filterPair(pair)]

#Filtrado de caracteres raros
def normalize(s):
    s = s.lower().strip()
    s = ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

def readData(lang1, lang2, path):
    lines = open(path, encoding='utf-8').read().strip().split('\n')
    print("Leyendo archivo")
    pairs = [[normalize(s) for s in l.split('\t')] for l in lines]
    inputL = Lang(lang1)
    outputL = Lang(lang2)
    return inputL, outputL, pairs

def prepareData(lang1, lang2, path):
    inputL, outputL, pairs = readData(lang1, lang2, path)
    pairs = filter(pairs)
    print("%s sentence pairs" % len(pairs))
    for pair in pairs:
        inputL.addSentence(pair[0])
        outputL.addSentence(pair[1])

    return inputL, outputL, pairs

### Usamos una arquitectura de tipo Encoder-Decoder usando GRUs con atención del tipo:
### $softmax(v^\intercal \tanh(Wq + Uk))$

In [4]:
#------------------------------------------------------------------Encoder
class EncoderGRU(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

#-----------------------------------------------------------------Attention
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        score = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        score = score.squeeze(2).unsqueeze(1)
        weights = F.softmax(score, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

#-----------------------------------------------------------------Decoder
class DecoderGRU(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(DecoderGRU, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = Attention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, enc_out, enc_h, target_tensor=None):
        batch_size = enc_out.size(0)
        dec_inp = torch.empty(batch_size, 1, dtype=torch.long,
                              device=device).fill_(BOS_token)
        dec_h = enc_h
        dec_out, attentions = [],[]

        for i in range(MAX_LENGTH):
            decoder_output, dec_h, attn_weights = self.forward_step(
                dec_inp, dec_h, enc_out)
            dec_out.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing
                dec_inp = target_tensor[:, i].unsqueeze(1)
            else:
                _, topi = decoder_output.topk(1)
                dec_inp = topi.squeeze(-1).detach()

        dec_out = torch.cat(dec_out, dim=1)
        dec_out = F.log_softmax(dec_out, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return dec_out, dec_h, attentions


    def forward_step(self, input, hidden, enc_out):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, enc_out)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [5]:
#DataLoader
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def get_dataloader(batch_size,dat):
    path = '/content/drive/MyDrive/translating/' + dat + '.txt'
    inputL, outputL, pairs = prepareData('eng', dat , path)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(inputL, inp)
        tgt_ids = indexesFromSentence(outputL, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return inputL, outputL, train_dataloader

### Entrenamos usando una función de pérdida de log-likelihood negativo.

In [6]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,decoder_optimizer, criterion):
    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        enc_out, enc_h = encoder(input_tensor)
        dec_out, _, _ = decoder(enc_out, enc_h, target_tensor)
        loss = criterion(
            dec_out.view(-1, dec_out.size(-1)),
            target_tensor.view(-1))
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader)

def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=10):
    loss_total = 0
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    for epoch in tqdm(range(1, n_epochs + 1)):
        loss = train_epoch(train_dataloader, encoder, decoder,
                           encoder_optimizer, decoder_optimizer, criterion)
        loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = loss_total / print_every
            loss_total = 0
            print("It: ",epoch,"-----> Loss: ", print_loss_avg)

def evaluate(encoder, decoder, sentence, inputL, outputL):
    with torch.no_grad():
        input_tensor = tensorFromSentence(inputL, sentence)

        enc_out, enc_h = encoder(input_tensor)
        dec_out, dec_h, decoder_attn = decoder(enc_out, enc_h)
        _, topi = dec_out.topk(1)
        decoded_ids = topi.squeeze()
        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(outputL.index2word[idx.item()])
    return decoded_words, decoder_attn

### Probamos entrenar Inglés a Alemán con 50 épocas y una longitud máxima de sencuencia de 10 como filtro.

In [ ]:
hidden_size = 128
batch_size = 32

inputL, outputL, train_dataloader = get_dataloader(batch_size,'deu')
encoder = EncoderGRU(inputL.n_words, hidden_size).to(device)
decoder = DecoderGRU(hidden_size, outputL.n_words).to(device)

train(train_dataloader, encoder, decoder, n_epochs = 50, print_every=5)

Leyendo archivo
223849 sentence pairs


  0%|          | 0/50 [00:00<?, ?it/s]

It:  5 -----> Loss:  1.4197242100283787
It:  10 -----> Loss:  0.818208046856439
It:  15 -----> Loss:  0.6748850814925731
It:  20 -----> Loss:  0.6021373375168251
It:  25 -----> Loss:  0.5568963250001545
It:  30 -----> Loss:  0.5243726876393395
It:  35 -----> Loss:  0.5000027440812093
It:  40 -----> Loss:  0.48095414767125944
It:  45 -----> Loss:  0.4651921991750879
It:  50 -----> Loss:  0.45234688633574693


In [8]:
def evaluateRan(encoder, decoder, pairs, n=30):
    for i in range(n):
        pair = random.choice(pairs)
        print("\n",pair[0])
        output_words, _ = evaluate(encoder, decoder, pair[0], inputL, outputL)
        output_sentence = ' '.join(output_words[:-1])
        print('Pred: ---> ', output_sentence)
        print('Real: ---> ', pair[1])

In [ ]:
path = '/content/drive/MyDrive/translating/deu.txt'
inputL, outputL, pairs = prepareData('eng', 'deu', path)

Leyendo archivo
223849 sentence pairs


### Evaluamos 15 sentencias al azar sobre el modelo y podemos observar que con 50 épocas el modelo es capaz de de traducir correctamente muchas de las frases del dataset, y en algunas en las que no lo hace exacto, su traducción sigue teniendo el mismo sentido utilizando diferentes palabras.

In [ ]:
encoder.eval(), decoder.eval()
evaluateRan(encoder, decoder, pairs)


 i have nothing else
Pred: --->  ich habe sonst nichts zu nichts <EOS>
Real: --->  ich habe sonst nichts

 don t touch the button
Pred: --->  beruhren sie nicht den knopf <EOS>
Real: --->  nicht den knopf anfassen !

 did you rent an apartment ?
Pred: --->  hattet ihr eine wohnung gemietet ? <EOS>
Real: --->  hast du eine wohnung gemietet ?

 she s putting the children to bed
Pred: --->  sie raumte die kinder ins bett <EOS>
Real: --->  sie bringt die kinder ins bett

 i m not safe here
Pred: --->  ich bin hier nicht sicher <EOS>
Real: --->  ich bin hier nicht sicher

 i said sit down
Pred: --->  ich sagte er soll mich mal setzen <EOS>
Real: --->  ich hab gesagt du sollst dich hinsetzen

 you re a funny girl
Pred: --->  du bist ein lustiges madchen ein madchen madchen <EOS>
Real: --->  du bist ein lustiges madchen

 the door will be painted tomorrow
Pred: --->  die tur wird morgen gestrichen werden <EOS>
Real: --->  die tur wird morgen gestrichen

 i think this will work
Pred: --->  ic


---
## Inglés - Español
### Ahora usamos los datos de Inglés-Español para entrenar bajo las mismas condiciones e hiperparámetros la arquitectura.



In [10]:
hidden_size = 128
batch_size = 32

inputL, outputL, train_dataloader = get_dataloader(batch_size,'spa')
encoder = EncoderGRU(inputL.n_words, hidden_size).to(device)
decoder = DecoderGRU(hidden_size, outputL.n_words).to(device)

train(train_dataloader, encoder, decoder, n_epochs = 50, print_every=5)

Leyendo archivo
119044 sentence pairs


  0%|          | 0/50 [00:00<?, ?it/s]

It:  5 -----> Loss:  1.5078448687853017
It:  10 -----> Loss:  0.7453241566373788
It:  15 -----> Loss:  0.5684872798952035
It:  20 -----> Loss:  0.4854022942909916
It:  25 -----> Loss:  0.4338874899141252
It:  30 -----> Loss:  0.3981941767595558
It:  35 -----> Loss:  0.37166602959622624
It:  40 -----> Loss:  0.350618623064526
It:  45 -----> Loss:  0.33331450056930406
It:  50 -----> Loss:  0.31957828291191015


In [12]:
path = '/content/drive/MyDrive/translating/spa.txt'
inputL, outputL, pairs = prepareData('eng', 'spa', path)

Leyendo archivo
138549 sentence pairs


In [29]:
encoder.eval(), decoder.eval()
evaluateRan(encoder, decoder, pairs)


 this tie doesn t go with my suit
Pred: --->  esta corbata no combina con mi traje
Real: --->  esta corbata no combina con mi traje

 that s what i said
Pred: --->  eso es lo que dije dije
Real: --->  eso fue lo que dije

 she works as a secretary in an office
Pred: --->  ella trabaja como secretaria en una oficina
Real: --->  ella trabaja como secretaria en una oficina

 the sky is clear and the sun is bright
Pred: --->  el cielo esta despejado y el sol brillante
Real: --->  el cielo esta despejado y el sol brillante

 she was busy
Pred: --->  estuvo ocupada estuve ocupado
Real: --->  ella estaba ocupada

 i ve made stew
Pred: --->  he hecho guiso a haber estofado
Real: --->  he hecho guiso

 have the police been called ?
Pred: --->  ha cambiado la policia ha llamado ?
Real: --->  se ha llamado a la policia ?

 i don t even know who i am anymore
Pred: --->  ya ni siquiera me doy soy yo
Real: --->  ya no me conozco

 he neither smokes nor drinks
Pred: --->  no fuma ni ni bebe ni bebe


### En la gran mayoría de pruebas realizadas la traducción tiene sentido, se encuentran algunos errores de redacción y cuando parafrasea lo hace de forma casi siempre correcta. A veces la secuencia traducida genera el token EOS tarde y el modelo general uno o máximo dos tokens repetidos.

In [27]:
prompt = "i like to play with you"
output_words, _ = evaluate(encoder, decoder, prompt, inputL, outputL)
print(' '.join(output_words[:-1]))

me gusta jugar con vos


### Por último, probamos entrenar usando un tamaño de secuencia máxima mayor. Aquí practicamente entran todas las secuencias disponibles en el dataset. Vemos que la pérdida baja mucho más rápido al usar secuencias más largas para entrenar.

In [11]:
hidden_size = 128
batch_size = 32

inputL, outputL, train_dataloader = get_dataloader(batch_size,'spa')
encoder = EncoderGRU(inputL.n_words, hidden_size).to(device)
decoder = DecoderGRU(hidden_size, outputL.n_words).to(device)

train(train_dataloader, encoder, decoder, n_epochs = 50, print_every=5)

Leyendo archivo
138549 sentence pairs


  0%|          | 0/50 [00:00<?, ?it/s]

It:  5 -----> Loss:  1.1408303275793859
It:  10 -----> Loss:  0.5906463992334128
It:  15 -----> Loss:  0.46345707290580035
It:  20 -----> Loss:  0.40154272713162875
It:  25 -----> Loss:  0.362647176687767
It:  30 -----> Loss:  0.3349560948738325
It:  35 -----> Loss:  0.3143407500905198
It:  40 -----> Loss:  0.2979931370387452
It:  45 -----> Loss:  0.28448245705309694
It:  50 -----> Loss:  0.2736660649512161


In [13]:
encoder.eval(), decoder.eval()
evaluateRan(encoder, decoder, pairs)


 it s time for us to get serious
Pred: --->  es hora de que pongamos correcto
Real: --->  es hora de tomarnoslo en serio

 don t blame yourself for what happened to tom
Pred: --->  no te culpes por lo que le paso a tom que la mayor
Real: --->  no te culpes por lo que le paso a tom

 what do you know about my country ?
Pred: --->  que sabes de mi pais ?
Real: --->  que sabes de mi pais ?

 we expected him to support us
Pred: --->  esperabamos que el nos apoyara
Real: --->  esperabamos que el nos apoyara

 the stars came out
Pred: --->  las estrellas salieron saliendo
Real: --->  las estrellas salieron

 my sister usually goes to the park every weekend
Pred: --->  mi hermana normalmente va al parque todos los fines de semana en el parque
Real: --->  mi hermana normalmente va al parque todos los fines de semana

 a detective arrived upon the scene of the crime
Pred: --->  un detective llego a la escena del crimen
Real: --->  un detective llego a la escena del crimen

 could you come to m

### Parece que la traducción mejora, pero la cantidad de tokens extra y repetidos aumenta considerablemente.